In [77]:
import warnings
warnings.simplefilter("ignore")

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import numpy as np
 
import torch
import torch.nn as nn
import datasets 
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import PIL
import torch.nn.functional as F
from PIL import Image
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader , Subset, random_split
from torchvision.datasets.utils import download_and_extract_archive
import tensorboard
import torchmetrics


import torchvision.models as models
from datasets import load_dataset, load_metric
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks import EarlyStopping

In [78]:
class CustomDataset(Dataset):
    def __init__(self, data, transform):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]['image']
        label = self.data[idx]['labels']
        image = self.transform(image)
        return image, label

In [79]:
class CatsVsDogsDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=16):
        super().__init__()
        self.batch_size = batch_size
        self.transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])
    
    def prepare_data(self):
        load_dataset('Bingsu/Cat_and_Dog', split='train')
        load_dataset('Bingsu/Cat_and_Dog', split='test')

    def setup(self,stage=None):
            train_data = load_dataset('Bingsu/Cat_and_Dog', split='train')
            train_dataset = CustomDataset(train_data, self.transform)
            train_size = int(len(train_dataset) * (0.8))
            val_size = len(train_dataset) - train_size
            self.train_dataset, self.val_dataset = random_split(train_dataset, [train_size, val_size])
            test_data = load_dataset('Bingsu/Cat_and_Dog', split='test')
            self.test_dataset = CustomDataset(test_data, self.transform)
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, num_workers=0, shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=0)
    
    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=0)

In [80]:
class SEBlock(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
            )
        
    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)


In [81]:
class ResNet50(pl.LightningModule):
    def __init__(self, lr=1e-4):
        super().__init__()
        self.lr = lr
        self.model = models.resnet50(pretrained=True)
        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 2)
        self.dropout = nn.Dropout(p=0.5)
        self.bn1 = nn.BatchNorm1d(1024)
        self.bn2 = nn.BatchNorm1d(512)
        self.se_block1 = SEBlock(1024)
        self.se_block2 = SEBlock(2048)

        for param in self.model.parameters():
            param.requires_grad = False
        self.all_val_losses = []
        
        
    def forward(self, x):
        x = self.model.conv1(x)
        x = self.model.bn1(x)
        x = self.model.relu(x)
        x = self.model.maxpool(x)

        x = self.model.layer1(x)
        x = self.model.layer2(x)
        x = self.model.layer3(x)
        x = self.se_block2(self.model.layer4(x))

        x = F.avg_pool2d(x, x.size()[3])
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.bn1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.bn2(x)
        x = self.fc3(x)
        return x

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.logger.log_metrics({'train_loss': loss}, step=self.global_step)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.all_val_losses.append(loss.item())
        self.logger.log_metrics({'val_loss': loss}, step=self.global_step)
        return loss
    
    def on_validation_epoch_end(self):
        avg_loss = torch.tensor(self.all_val_losses).mean()
        self.logger.log_metrics({'val_loss': avg_loss}, step=self.current_epoch)
        del avg_loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('test_loss', loss)
        preds = torch.argmax(y_hat, dim=1)
        acc = torch.sum(preds == y) / float(len(y))
        self.logger.log_metrics({'test_loss': loss, 'test_acc': acc}, step=self.global_step)
    
    def save_model(self, filepath):
        torch.save(self.state_dict(), filepath)

    def load_model(self, filepath):
        self.load_state_dict(torch.load(filepath))
        

In [82]:
 %reload_ext tensorboard

In [83]:
%load_ext tensorboard
%tensorboard --logdir=log/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 15980), started 19:11:43 ago. (Use '!kill 15980' to kill it.)

In [ ]:
weights_file_path = "resnet50_weights.pt"
model = ResNet50()
#model.load_state_dict(torch.load(weights_file_path))
#model = torch.compile(model)
logger = TensorBoardLogger('log/', name='ResNet50')
trainer = pl.Trainer(accelerator='auto',max_epochs=20, deterministic=True, logger=logger)
dm = CatsVsDogsDataModule()
dm.setup()
trainer.fit(model, datamodule=dm)
model.save_model(weights_file_path)
trainer.test(model, dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Found cached dataset parquet (C:/Users/User/.cache/huggingface/datasets/Bingsu___parquet/Bingsu--Cat_and_Dog-700815090bea8354/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (C:/Users/User/.cache/huggingface/datasets/Bingsu___parquet/Bingsu--Cat_and_Dog-700815090bea8354/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (C:/Users/User/.cache/huggingface/datasets/Bingsu___parquet/Bingsu--Cat_and_Dog-700815090bea8354/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (C:/Users/User/.cache/huggingface/datasets/Bingsu___parquet/Bingsu--Cat_and_Dog-700815090bea8354/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (C:/Users/User/.cache/huggingfac

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Found cached dataset parquet (C:/Users/User/.cache/huggingface/datasets/Bingsu___parquet/Bingsu--Cat_and_Dog-700815090bea8354/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (C:/Users/User/.cache/huggingface/datasets/Bingsu___parquet/Bingsu--Cat_and_Dog-700815090bea8354/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (C:/Users/User/.cache/huggingface/datasets/Bingsu___parquet/Bingsu--Cat_and_Dog-700815090bea8354/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Found cached dataset parquet (C:/Users/User/.cache/huggingface/datasets/Bingsu___parquet/Bingsu--Cat_and_Dog-700815090bea8354/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


Testing: 0it [00:00, ?it/s]